In [12]:
import numpy as np
import pandas as pd
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image


In [13]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = self.annotations.iloc[idx, 0]
        label = int(self.annotations.iloc[idx, 1])
        img_path = os.path.join(self.img_dir, img_name)

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [20]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = CustomImageDataset(
    csv_file="data/train.csv",
    img_dir="data/train",
    transform=transform
)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

dataiter = iter(dataloader)
images, labels = next(dataiter)



FileNotFoundError: [Errno 2] No such file or directory: 'data/train\\fda9a27561a06f32a3a4779686a04ebf.jpg'